In [1]:
def matchdata (x):
    

    # Import necessary libraries.

    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    from urllib.request import urlopen
    import re
    from datetime import datetime, time


    # Get the url of the match from the function input.

    url = x


    # BeautifulSoup

    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")


    ## Main data of match.

    teamshtml = soup.find_all("div", {"class":"scorebox"})
    matchhtml = soup.find_all("div", {"class":"scores"})

    team = []
    location = ["Home", "Away"]
    score = []
    xg = []

    for i in range(2):
        team.append(teamshtml[0].findAll("a")[(0+(i*4))].text)

    for i in range(2):
        score.append(matchhtml[i].findAll("div")[0].text)

    for i in range(2):
        xg.append(matchhtml[i].findAll("div")[1].text)

    date_string = "Sunday September 2, 2018"

    # Define the format of the date string
    date_format = "%A %B %d, %Y"

    # Parse the string into a datetime object
    date_object = datetime.strptime(soup.find_all("div", {"class":"scorebox_meta"})[0].find_all("a")[0].text, date_format)

    matchdata = {}
    matchdata["team"] = team
    matchdata["matchday"] = ((re.search(r'\((.*?)\)', soup.find_all("div", {"class":"scorebox_meta"})[0].find_all("div")[1].text)).group(1).split(" ")[1])
    matchdata["date"] = datetime.strptime(soup.find_all("div", {"class":"scorebox_meta"})[0].find_all("a")[0].text, date_format)
    matchdata["time"] = soup.find_all("div", {"class":"scorebox_meta"})[0].find_all("div")[0].find_all("span")[0].text.replace(" (venue time)", "")
    matchdata["location"] = location
    matchdata["opponent"] = reversed(team)
    matchdata["score"] = score
    matchdata["score_against"] = reversed(score)
    matchdata["xg_against"] = reversed(xg)

    data_of_match = pd.DataFrame(matchdata)

    data_of_match4 = data_of_match[data_of_match.columns[:6]].replace("", "0")
    data_of_match5 = data_of_match[data_of_match.columns[6:]].replace("", "0").astype("float")
    data_of_match = pd.concat([data_of_match4, data_of_match5], axis=1)

    data_of_match['time'] = data_of_match['time'].apply(lambda x: datetime.strptime(x, '%H:%M').time())
    data_of_match['matchday'] = data_of_match['matchday'].astype("int")
    data_of_match['score'] = data_of_match['score'].astype("int")
    data_of_match['score_against'] = data_of_match['score_against'].astype("int")


    # All Player's Statistics

    to_concat = range(3, 9)
    home_player_statistics = []

    for k in to_concat:
        lengthoftable = []
        allhtml = soup.find_all("table")[k]

        for i in allhtml.findAll(attrs={"data-stat": True}):
            if ("header" not in i['data-stat']) and len(i['data-stat']) > 0:
                if re.search(r'scope="col"', str(i)):
                    lengthoftable.append(i)

        length = int(len(lengthoftable))

        table = []
        for i in allhtml.findAll(attrs={"data-stat": True}):
            if ("header" not in i['data-stat']) and len(i['data-stat']) > 0:
                table.append(i.text)
            
        lists_of_five = [[] for _ in range(length)]

        for index, value in enumerate(table[:-length]):
            list_index = index % length
            lists_of_five[list_index].append(value.replace("\xa0\xa0\xa0", ""))

        df = pd.DataFrame(lists_of_five).T
        df.columns = df.iloc[0]
        df = df.drop(0)
        df.insert(loc=0, column='team', value=str(soup.find_all("table")[k].find_all("caption")[0].text.replace(" Player Stats Table", "")))
        
        if k > 3:
            new = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6]],axis = 1)

        if len(home_player_statistics) == 0:
            home_player_statistics = df
        else:
            home_player_statistics = pd.concat([home_player_statistics, new], axis=1)
        
    def rename_duplicate_columns(df):
        cols = pd.Series(df.columns)
        for dup in cols[cols.duplicated()].unique():
            cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
        df.columns = cols
        
    rename_duplicate_columns(home_player_statistics)

    to_concat = range(10, 16)
    away_player_statistics = []

    for k in to_concat:
        lengthoftable = []
        allhtml = soup.find_all("table")[k]

        for i in allhtml.findAll(attrs={"data-stat": True}):
            if ("header" not in i['data-stat']) and len(i['data-stat']) > 0:
                if re.search(r'scope="col"', str(i)):
                    lengthoftable.append(i)

        length = int(len(lengthoftable))

        table = []
        for i in allhtml.findAll(attrs={"data-stat": True}):
            if ("header" not in i['data-stat']) and len(i['data-stat']) > 0:
                table.append(i.text)
            
        lists_of_five = [[] for _ in range(length)]

        for index, value in enumerate(table[:-length]):
            list_index = index % length
            lists_of_five[list_index].append(value.replace("\xa0\xa0\xa0", ""))

        df = pd.DataFrame(lists_of_five).T
        df.columns = df.iloc[0]
        df = df.drop(0)
        df.insert(loc=0, column='team', value=str(soup.find_all("table")[k].find_all("caption")[0].text.replace(" Player Stats Table", "")))
        
        if k > 3:
            new = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6]],axis = 1)

        if len(away_player_statistics) == 0:
            away_player_statistics = df
        else:
            away_player_statistics = pd.concat([away_player_statistics, new], axis=1)
        
    def rename_duplicate_columns(df):
        cols = pd.Series(df.columns)
        for dup in cols[cols.duplicated()].unique():
            cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
        df.columns = cols
        
    rename_duplicate_columns(away_player_statistics)

    player_statistics = pd.concat([home_player_statistics, away_player_statistics])
    player_statistics = player_statistics.reset_index().drop(["index"], axis=1)

    player_statistics5 = player_statistics[player_statistics.columns[:6]].replace("", "0")
    player_statistics6 = player_statistics[player_statistics.columns[6:]].replace("", "0").astype("float")
    player_statistics = pd.concat([player_statistics5, player_statistics6], axis=1)

    player_statistics.drop(["PK", "PKatt", "Min", "Gls"], axis=1, inplace=True)
    player_statistics.drop(["Player", "#", "Nation", "Pos", "Age", "Att", "Att_1", "Succ", "Blocks_1"], axis=1, inplace=True)
    player_statistics.drop(["xG"], axis=1, inplace=True)
    player_statistics.drop(["Cmp_1", "Att_2", "Cmp%_1", "TotDist"], axis=1, inplace=True)
    player_statistics.drop(["Cmp_2", "Att_3", "Cmp%_2", "Cmp_3", "Att_4", "Cmp%_3", "Cmp_4", "Att_5", "Cmp%_4", "Ast_1", "xAG_1"], axis=1, inplace=True)
    player_statistics.drop(["PrgP_1", "Att_6", "Live", "Dead", "FK"], axis=1, inplace=True)
    player_statistics.drop(["Sw", "TI", "In", "Out", "Str", "Cmp_5", "Tkl_1"], axis=1, inplace=True)
    player_statistics.drop(["Def 3rd", "Mid 3rd", "Att 3rd"], axis=1, inplace=True)
    player_statistics.drop(["Tkl%", "Blocks_2", "Sh_1", "Pass", "Int_1", "Tkl+Int"], axis=1, inplace=True)
    player_statistics.drop(["Touches_1", "Def Pen", "Def 3rd_1", "Mid 3rd_1", "Att 3rd_1", "Live_1", "Att_8", "Succ_1", "Succ%"], axis=1, inplace=True)
    player_statistics.drop(["Carries_1", "TotDist_1", "PrgDist_1", "PrgC_1", "1/3_1"], axis=1, inplace=True)
    player_statistics.drop(["Rec", "PrgR", "CrdY_1", "CrdR_1"], axis=1, inplace=True)
    player_statistics.drop(["Crs_1", "Int_2", "TklW_1", "Fld", "Won%", "Tkld%", "Cmp%"], axis=1, inplace=True)

    player_statistics.rename(columns={"Gls": "goals"}, inplace=True)
    player_statistics.rename(columns={"Ast": "assists"}, inplace=True)
    player_statistics.rename(columns={"Sh": "shots"}, inplace=True)
    player_statistics.rename(columns={"SoT": "shots_on_target"}, inplace=True)
    player_statistics.rename(columns={"CrdY": "yellows"}, inplace=True)
    player_statistics.rename(columns={"CrdR": "reds"}, inplace=True)
    player_statistics.rename(columns={"Touches": "touches"}, inplace=True)
    player_statistics.rename(columns={"Tkl": "tackles"}, inplace=True)
    player_statistics.rename(columns={"Int": "interceptions"}, inplace=True)
    player_statistics.rename(columns={"Blocks": "blocks_by_player"}, inplace=True)
    player_statistics.rename(columns={"npxG": "non_penalty_expected_goals"}, inplace=True)
    player_statistics.rename(columns={"xAG": "expected_assisted_goals"}, inplace=True)
    player_statistics.rename(columns={"SCA": "shot_creating_actions"}, inplace=True)
    player_statistics.rename(columns={"GCA": "goal_creating_actions"}, inplace=True)
    player_statistics.rename(columns={"Cmp": "passes_completed"}, inplace=True)
    player_statistics.rename(columns={"PrgP": "progressive_passes"}, inplace=True)
    player_statistics.rename(columns={"Carries": "carries"}, inplace=True)
    player_statistics.rename(columns={"PrgC": "progressive_carries"}, inplace=True)
    player_statistics.rename(columns={"PrgP": "progressive_passes"}, inplace=True)
    player_statistics.rename(columns={"PrgDist": "progressive_passing_distance"}, inplace=True)
    player_statistics.rename(columns={"xA": "expected_assists"}, inplace=True)
    player_statistics.rename(columns={"KP": "key_passes"}, inplace=True)
    player_statistics.rename(columns={"1/3": "passes_into_final_third"}, inplace=True)
    player_statistics.rename(columns={"PPA": "passes_into_penalty_area"}, inplace=True)
    player_statistics.rename(columns={"CrsPA": "crosses_into_penalty_area"}, inplace=True)
    player_statistics.rename(columns={"TB": "through_balls"}, inplace=True)
    player_statistics.rename(columns={"Crs": "crosses"}, inplace=True)
    player_statistics.rename(columns={"CK": "corner_kicks"}, inplace=True)
    player_statistics.rename(columns={"Off": "passes_offside"}, inplace=True)
    player_statistics.rename(columns={"Blocks": "blocks_by_opponent"}, inplace=True)
    player_statistics.rename(columns={"TklW": "tackles_won"}, inplace=True)
    player_statistics.rename(columns={"Tkl_2": "dribblers_tackled"}, inplace=True)
    player_statistics.rename(columns={"Att_7": "dribbler_tackles_attempts"}, inplace=True)
    player_statistics.rename(columns={"Lost": "unsuccessful_dribbler_challenge"}, inplace=True)
    player_statistics.rename(columns={"Clr": "clearances"}, inplace=True)
    player_statistics.rename(columns={"Err": "error_leading_to_opponent_shot"}, inplace=True)
    player_statistics.rename(columns={"Att Pen": "touches_in_attacking_penalty_area"}, inplace=True)
    player_statistics.rename(columns={"Tkld": "player_take_ons_tackled"}, inplace=True)
    player_statistics.rename(columns={"CPA": "carries_into_penalty_box"}, inplace=True)
    player_statistics.rename(columns={"Mis": "failed_controls"}, inplace=True)
    player_statistics.rename(columns={"Dis": "dispossession_after_tackle"}, inplace=True)
    player_statistics.rename(columns={"2CrdY": "double_yellows"}, inplace=True)
    player_statistics.rename(columns={"Fls": "fouls"}, inplace=True)
    player_statistics.rename(columns={"2CrdY": "double_yellows"}, inplace=True)
    player_statistics.rename(columns={"Off_1": "offsides"}, inplace=True)
    player_statistics.rename(columns={"PKwon": "penalty_kicks_won"}, inplace=True)
    player_statistics.rename(columns={"PKcon": "penalty_kicks_conceded"}, inplace=True)
    player_statistics.rename(columns={"OG": "own_goals"}, inplace=True)
    player_statistics.rename(columns={"Recov": "recoveries"}, inplace=True)
    player_statistics.rename(columns={"Won": "aerials_won"}, inplace=True)
    player_statistics.rename(columns={"Lost_1": "aerials_lost"}, inplace=True)

    player_statistics = player_statistics.groupby("team").sum().reset_index()
    player_statistics.drop(["team"], axis=1, inplace=True)


    # Match's Main Plays and Events.

    lengthoftable = []
    allhtml = soup.find_all("table")[17]

    for i in allhtml.findAll(attrs={"data-stat": True}):
        if ("header" not in i['data-stat']) and len(i['data-stat']) > 0:
            if re.search(r'scope="col"', str(i)):
                lengthoftable.append(i)

    length = int(len(lengthoftable))

    table = []
    for i in allhtml.findAll(attrs={"data-stat": True}):
        if ("header" not in i['data-stat']) and len(i['data-stat']) > 0:
            table.append(i.text)
        
    lists_of_five = [[] for _ in range(length)]

    for index, value in enumerate(table[:-length]):
        list_index = index % length
        lists_of_five[list_index].append(value.replace("\xa0\xa0\xa0", ""))

    match_events = pd.DataFrame(lists_of_five).T
    match_events.columns = match_events.iloc[0]
    match_events.drop(0, inplace=True)
    match_events.drop(["Player", "Minute", "Outcome", "Body Part", "Notes", "Event"], axis=1, inplace=True)
    match_events.replace("", "0", inplace=True)
    match_events = match_events[match_events["Squad"] != "0"]
    match_events["Distance"] = match_events["Distance"].astype("int")
    match_events["xG"] = match_events["xG"].astype("float")
    match_events["PSxG"] = match_events["PSxG"].astype("float")
    match_events = match_events.iloc[:, [0, 1, 2]]
    match_events.rename(columns={"Squad": "team"}, inplace=True)

    match_events = match_events.groupby("team").agg({"xG":"sum", "PSxG":"mean"}).reset_index()
    match_events.drop(["team"], axis=1, inplace=True)

    
    # Concatenate all the previous dataframes.

    the_match_full_data = pd.concat([data_of_match, match_events, player_statistics], axis=1)

    return the_match_full_data

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import re
from datetime import datetime, time
import time

url = "https://fbref.com/en/comps/12/2023-2024/schedule/2023-2024-La-Liga-Scores-and-Fixtures"

res = requests.get(url)
soup = BeautifulSoup(res.content, "html.parser")

In [3]:
i = 0
links = []

while True:
    if len(soup.find_all("td", class_="left", attrs={"data-stat": "match_report"})[i].find_all("a")) > 0:
        links.append("https://fbref.com"+soup.find_all("td", class_="left", attrs={"data-stat": "match_report"})[i].find_all("a")[0]["href"])
    i += 1

IndexError: list index out of range

In [4]:
len(links)

380

In [5]:
links = links[:190]

In [6]:
zzz = []

for i in links:
    time.sleep(5)
    if len(zzz) == 0:
        zzz = matchdata(i)
    else:
        zzz = pd.concat([zzz, matchdata(i)])
    print(f"{round((len(zzz)/190)*100, 2)}%")

1.05%
2.11%
3.16%
4.21%
5.26%
6.32%
7.37%
8.42%
9.47%
10.53%
11.58%
12.63%
13.68%
14.74%
15.79%
16.84%
17.89%
18.95%
20.0%
21.05%
22.11%
23.16%
24.21%
25.26%
26.32%
27.37%
28.42%
29.47%
30.53%
31.58%
32.63%
33.68%
34.74%
35.79%
36.84%
37.89%
38.95%
40.0%
41.05%
42.11%
43.16%
44.21%
45.26%
46.32%
47.37%
48.42%
49.47%
50.53%
51.58%
52.63%
53.68%
54.74%


In [ ]:
zzz.sample()

NameError: name 'zzz' is not defined

In [ ]:
zzz = zzz[zzz["team"]!="Prev Match"]
zzz = zzz[zzz["opponent"]!="Prev Match"]
zzz

,team,matchday,date,time,location,opponent,score,score_against,xg_against,xG,...,dispossession_after_tackle,double_yellows,fouls,offsides,penalty_kicks_won,penalty_kicks_conceded,own_goals,recoveries,aerials_won,aerials_lost
0,Almería,1,2023-08-11,19:30:00,Home,Rayo Vallecano,0,2,2.1,1.35,...,3.0,0.0,16.0,1.0,0.0,2.0,0.0,47.0,11.0,18.0
1,Rayo Vallecano,1,2023-08-11,19:30:00,Away,Almería,2,0,1.4,2.08,...,9.0,0.0,25.0,1.0,1.0,0.0,0.0,41.0,18.0,11.0
0,Sevilla,1,2023-08-11,22:00:00,Home,Valencia,1,2,1.1,0.71,...,11.0,0.0,15.0,3.0,0.0,0.0,0.0,58.0,17.0,19.0
1,Valencia,1,2023-08-11,22:00:00,Away,Sevilla,2,1,0.7,1.12,...,16.0,0.0,17.0,0.0,0.0,0.0,0.0,59.0,19.0,17.0
0,Real Sociedad,1,2023-08-12,17:00:00,Home,Girona,1,1,0.8,0.81,...,10.0,0.0,9.0,3.0,0.0,0.0,0.0,45.0,18.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Barcelona,7,2023-09-26,21:30:00,Away,Mallorca,2,2,0.8,0.66,...,8.0,0.0,19.0,6.0,0.0,0.0,0.0,63.0,20.0,19.0
0,Athletic Club,7,2023-09-27,19:00:00,Home,Getafe,2,2,1.4,0.75,...,4.0,0.0,9.0,6.0,0.0,0.0,0.0,61.0,18.0,10.0
1,Getafe,7,2023-09-27,19:00:00,Away,Athletic Club,2,2,0.7,1.34,...,7.0,0.0,19.0,1.0,0.0,0.0,0.0,58.0,10.0,18.0
0,Real Madrid,7,2023-09-27,19:00:00,Home,Las Palmas,2,0,0.7,0.65,...,8.0,0.0,9.0,2.0,0.0,0.0,0.0,53.0,3.0,5.0


In [ ]:
zzz.to_csv("../tables/fbref.com/23-24.csv")